In [1]:
import tensorflow as tf
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image

import cv2
from matplotlib import pyplot as plt

2024-10-02 02:57:36.436355: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:

def get_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    return (x_train, y_train, x_test, y_test)

In [3]:
threshold=100
def on_capture(x):
    global threshold
    threshold=x

In [4]:
mouse_down=False
def clicked(event, x, y, flags, params):
    global mouse_down
    if event == cv2.EVENT_LBUTTONDOWN:
        mouse_down= not mouse_down


In [5]:
# model

def train_model(x_train,x_test,y_train,y_test):

    class CustomCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self,epoch,logs={}):
            print(logs)
            if(logs.get('accuracy')>=0.99):
                print("Cancelling futher training, Accuracy of 99% reached")
                self.model.stop_training=True

    x_train=x_train/255
    x_test=x_test/255

    model=tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1),activation=tf.nn.relu),
        tf.keras.layers.MaxPool2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(64,kernel_size=(3,3), input_shape=(28,28,1),activation=tf.nn.relu),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128,activation=tf.nn.relu),
        tf.keras.layers.Dense(10,activation=tf.nn.softmax)
    ])

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    
    training=model.fit(x_train, y_train, epochs=10, batch_size=32, callbacks=[CustomCallback()])
    print(training.epoch, training.history['accuracy'][-1])  

    return model           

In [6]:
def predict(model,image):
    imgs = np.array([image])
    res = model.predict(imgs)
    index = np.argmax(res)
    return str(index)

In [7]:
def cv_video(model):
    global threshold
    
    video_frame=cv2.namedWindow('my_window', cv2.WINDOW_NORMAL)
    video_capture = cv2.VideoCapture(cv2.CAP_V4L2)
    cv2.setMouseCallback('my_window', clicked)
    cv2.createTrackbar('threshold','my_window',100,255,on_capture)
    my_window=np.zeros((480, 640),np.uint8)
    framecount=0

    while True:
        frame_captured,video_frame=video_capture.read()
        if(mouse_down):

            framecount+=1

            video_frame[0:480, 0:80] = 0
            video_frame[0:480, 560:640] = 0

            grayFrame = cv2.cvtColor(video_frame, cv2.COLOR_BGR2GRAY)
            _, thr = cv2.threshold(grayFrame, threshold, 255, cv2.THRESH_BINARY_INV)

            resizedFrame = thr[240-75:240+75, 320-75:320+75]
            my_window[240-75:240+75, 320-75:320+75] = resizedFrame

            iconImg = cv2.resize(resizedFrame, (28, 28))

            result = predict(model,iconImg)
            if framecount == 5:
                my_window[0:480, 0:80] = 0
                framecount = 0

            cv2.putText(my_window, result, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 3)
            cv2.rectangle(my_window, (320-80, 240-80), (320+80, 240+80), (255, 255, 255), thickness=3)
            cv2.imshow('my_window', my_window)
        else:
            # display normal video 
            cv2.imshow('my_window', video_frame)

        # cv2.imshow('resized', resizedFrame)
        if cv2.waitKey(1) & 0xff == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

In [8]:
def load_model(model_path):
    """Loads a model from the specified path."""
    try:
        return tf.keras.models.load_model(model_path)
    except (FileNotFoundError, OSError):
        print("Model not found. Training a new model.")
        return None


In [9]:
def train_and_save_model(model_path):
    """Trains a new model, saves it to the specified path, and returns it."""
    x_train, y_train, x_test, y_test = get_data()
    model = train_model(x_train, x_test, y_train, y_test)
    model.save(model_path)
    return model

In [10]:
def main():
    model_path = 'model.sav'
    model = load_model(model_path)
    print(model is None)
    if model is None:
        model = train_and_save_model(model_path)
    cv_video(model)


In [ ]:
if __name__ == '__main__':
    main()